In [0]:
CREATE OR REPLACE TABLE IDENTIFIER(:catalog || '.' || :schema || '.sections') AS
WITH exploded_elements AS (
  SELECT
    p.path,
    element:id::STRING AS element_id,
    element:type::STRING AS element_type,
    element:content::STRING AS element_content
  FROM IDENTIFIER(:catalog || '.' || :schema || '.' || :parsed_table)  AS p,
  LATERAL explode(cast(p.parsed:document:elements AS ARRAY<VARIANT>)) AS t(element)
  WHERE element:id IS NOT NULL
),
section_headers AS (
  SELECT
    path,
    element_id,
    element_content,
    ROW_NUMBER() OVER (PARTITION BY path ORDER BY element_id::INT) AS mono_section_id
  FROM exploded_elements
  WHERE element_type = 'section_header'
),
section_tracking AS (
  SELECT
    e.path,
    e.element_id,
    e.element_type,
    e.element_content,
    sh.mono_section_id AS section_id,
    sh.element_content AS section_title
  FROM exploded_elements e
  LEFT JOIN LATERAL (
    SELECT mono_section_id, element_content
    FROM section_headers sh
    WHERE sh.path = e.path AND sh.element_id::INT <= e.element_id::INT
    ORDER BY sh.element_id::INT DESC
    LIMIT 1
  ) sh ON TRUE
)
SELECT
  path,
  section_id,
  section_title,
  CONCAT_WS('\n', COLLECT_LIST(element_content)) AS text,
  CONCAT_WS('\n', section_title, COLLECT_LIST(element_content)) AS combined_text
FROM section_tracking
WHERE element_type = 'text' 
  AND section_id IS NOT NULL
GROUP BY path, section_id, section_title
ORDER BY path, section_id;